In [ ]:
import pandas as pd
import tensorflow as tf
import keras as ks
import numpy as np

from keras.layers import Dense, Dropout, Conv2D, Flatten, Activation,MaxPooling2D
from keras.models import Sequential

import matplotlib.pylab as plt